In [ ]:
df.isnull().sum()

id                      0
public_petition_text    0
reason_category         0
dtype: int64

## Обработка данных

#### Необходимо почистить пропуски, сделать классическую обработку данных

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sw = nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

# очистка текста 
def clear(text):
    text = text.lower()
    text = re.sub(r'[^а-яё ]', "", text)
    text = nltk.tokenize.word_tokenize(text, language="russian")
    text = [morph.parse(word=i)[0].normal_form for i in text]
    text = [i for i in text if (i not in sw) & (len(i)>3)]
    return " ".join(text)

In [ ]:
# обработка текста
df["clear_text"] = df["public_petition_text"].apply(clear)

In [ ]:
df

,id,public_petition_text,reason_category,clear_text
0,3168490,снег на дороге,Благоустройство,снег дорога
1,3219678,очистить кабельный киоск от рекламы,Благоустройство,очистить кабельный киоск реклама
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство,просить убрать дерево кустарник который выйти ...
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД,неудовлетворительный состояние парадный надпис...
4,3336285,Граффити,Благоустройство,граффити
...,...,...,...,...
59884,3128111,прошу закрасить граффити,Благоустройство,просить закрасить граффити
59885,3276713,Прошу вас отремонтировать пешеходную дорожку,Благоустройство,просить отремонтировать пешеходный дорожка
59886,3274663,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...,необходимо демонтировать незаконный рекламный ...
59887,3359308,Очень гремит на ветру металлическая часть окна...,Кровля,очень греметь ветер металлический часть окно п...


## Векторизация

In [ ]:
# count векторизатор
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(ngram_range=(1, 3), max_features=1_000)
count_d = count.fit_transform(df["clear_text"])

In [ ]:
pd.DataFrame(count_d.todense(), columns=count.get_feature_names_out())

,аварийный,автомобиль,автомобильный,административный,административный ответственность,административный правонарушение,административный правонарушение санктпетербург,администрация,адрес,адресный,...,щель,щиток,эксплуатация,элемент,этаж,этаж парадный,этаж подъезд,являться,январь,ящик
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59884,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59885,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
count_df = pd.DataFrame(count_d.todense(), columns=count.get_feature_names_out())
new_df = pd.concat([df, count_df], axis=1)

In [ ]:
new_df 

,id,public_petition_text,reason_category,clear_text,аварийный,автомобиль,автомобильный,административный,административный ответственность,административный правонарушение,...,щель,щиток,эксплуатация,элемент,этаж,этаж парадный,этаж подъезд,являться,январь,ящик
0,3168490,снег на дороге,Благоустройство,снег дорога,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3219678,очистить кабельный киоск от рекламы,Благоустройство,очистить кабельный киоск реклама,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство,просить убрать дерево кустарник который выйти ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД,неудовлетворительный состояние парадный надпис...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,3336285,Граффити,Благоустройство,граффити,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59884,3128111,прошу закрасить граффити,Благоустройство,просить закрасить граффити,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59885,3276713,Прошу вас отремонтировать пешеходную дорожку,Благоустройство,просить отремонтировать пешеходный дорожка,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59886,3274663,Необходимо демонтировать незаконную рекламную ...,Незаконная информационная и (или) рекламная ко...,необходимо демонтировать незаконный рекламный ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59887,3359308,Очень гремит на ветру металлическая часть окна...,Кровля,очень греметь ветер металлический часть окно п...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
